In [121]:
import xml.etree.ElementTree as ET 
import csv
#xmlfile = '../IR_datasets/datascience.stackexchange.com/PostLinks.xml'
xmlfile = '../IR_datasets/biology.stackexchange.com/PostLinks.xml'
tree = ET.parse(xmlfile)

In [122]:
root = tree.getroot()

In [123]:
def createCSV(root,postlinksFile):
    with open(postlinksFile,mode='w',encoding='utf-8') as postlinks:
        postlinks = csv.writer(postlinks, delimiter=',')
        postlinks.writerow(['Id','PostId','RelatedPostId','LinkTypeId'])
        for row in root.findall('row'):
            Id = row.get('Id')
            postId = row.get('PostId')
            relatedpostId = row.get('RelatedPostId')
            linktypeId = row.get('LinkTypeId')
            postlinks.writerow([Id,postId,relatedpostId,linktypeId])


In [124]:
createCSV(root,"../csv_files/postlinks.csv")

In [125]:
import pandas as pd
postlinksDF = pd.read_csv('../csv_files/postlinks.csv')

In [126]:
type(postlinksDF)

pandas.core.frame.DataFrame

In [127]:
postlinksDF.shape

(5292, 4)

In [128]:
#final list of dict to keep track of related post ids and answer ids for each question id
postLinks = []
#to keep track of questions that were already encountered
posts = [] 

for index, row in postlinksDF.iterrows():
    temp = {'postID': None,'relatedID': [],'answerID': [] } 
    temp['postID'] = row['PostId']
    if(row['PostId'] in posts and len(posts)!=0):
        for i in range(0,len(posts)):
            if(row['PostId'] == postLinks[i]['postID']):
                postLinks[i]['relatedID'].append(row["RelatedPostId"])
                postLinks[i]['answerID'].append(row["Id"])
    else:  
        temp['relatedID'].append(row["RelatedPostId"])
        temp['answerID'].append(row["Id"])   
        posts.append(temp['postID'])
        postLinks.append(temp)       


In [129]:
len(postLinks)

3579

In [3]:
import pickle

#load the file of recommendations made by our model for each question

#Normalised Datascience 
n_ds_onlytitle_recommended_que= pickle.load(open("../models/n_ds_onlytitle_ques_recommendations.dat","rb"))
n_ds_withouttags_recommended_que= pickle.load(open("../models/n_ds_withouttags_ques_recommendations.dat","rb"))
n_ds_withtags_recommended_que= pickle.load(open("../models/n_ds_withtags_ques_recommendations.dat","rb"))
n_ds_ans = pickle.load(open("../models/n_ds_answers_recommendations.dat","rb"))

#Normalised Biology
n_bio_onlytitle_recommended_que= pickle.load(open("../models/n_bio_onlytitle_ques_recommendations.dat","rb"))
n_bio_withouttags_recommended_que= pickle.load(open("../models/n_bio_withouttags_ques_recommendations.dat","rb"))
n_bio_withtags_recommended_que= pickle.load(open("../models/n_bio_withtags_ques_recommendations.dat","rb"))
n_bio_ans = pickle.load(open("../models/n_bio_answers_recommendations.dat","rb"))

In [4]:
#Normalised Biology Combined (Q+A)
bio_combined_onlytitle= pickle.load(open("../models/bio_que_ans_onlytitle_recommendations.dat","rb"))
bio_combined_withouttags= pickle.load(open("../models/bio_que_ans_withouttags_recommendations.dat","rb"))
bio_combined_withtags= pickle.load(open("../models/bio_que_ans_withtags_recommendations.dat","rb"))

#Normalised Data Science Combined (Q+A)
ds_combined_onlytitle= pickle.load(open("../models/ds_que_ans_onlytitle_recommendations.dat","rb"))
ds_combined_withouttags= pickle.load(open("../models/ds_que_ans_withouttags_recommendations.dat","rb"))
ds_combined_withtags= pickle.load(open("../models/ds_que_ans_withtags_recommendations.dat","rb"))

In [116]:
#Function to evaluate recommendations for a question
#Evaluation made against the questions present in PostLinks.xml
#Weighted average of the number of questions from relatedID list are present in the list of recommendations 
#and the number of questions from relatedID list that are present in the top half of the list of recommendations

def evaluate(recommended,n):
    #print(len(recommended))
    score  = 0
    for i in range(0,len(postLinks)):
        present = 0 
        top = 0
        count = len(postLinks[i]['relatedID'])
        if postLinks[i]['postID'] in recommended.keys():
            idx = 0
            for rec in recommended[postLinks[i]['postID']][0:n]:
                #print(rec)
                idx+=1
                if rec[0] in postLinks[i]['relatedID']:
                    present+=1
                    if (idx >= int(len(postLinks[i]['relatedID'])/2)):
                        top+=1
            if(present):
                score+= (0.3* (present/count)+ 0.7*(top/present))
    return (score/len(postLinks))
    

In [120]:
print(evaluate(n_ds_onlytitle_recommended_que,30))
print(evaluate(n_ds_withouttags_recommended_que,30))
print(evaluate(n_ds_withtags_recommended_que,30))
print(evaluate(n_ds_ans,30))

print(evaluate(ds_combined_onlytitle,30))
print(evaluate(ds_combined_withtags,30))
print(evaluate(ds_combined_withouttags,30))

0.07424882629107982
0.07397887323943661
0.13045606975184434
0.07032863849765258
0.09507042253521122
0.13252347417840366
0.11089201877934265


In [130]:
print(evaluate(n_bio_onlytitle_recommended_que,30))
print(evaluate(n_bio_withouttags_recommended_que,30))
print(evaluate(n_bio_withtags_recommended_que,30))
print(evaluate(n_bio_ans,30))

print(evaluate(bio_combined_onlytitle,30))
print(evaluate(bio_combined_withouttags,30))
print(evaluate(bio_combined_withtags,30))

0.09297658962998455
0.13828856158277827
0.18910027408560573
0.1340232374033718
0.18438134488218347
0.19836569805346027
0.22919600447052285
